In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.simplefilter('ignore')

from os import path
import sys
sys.path.append(path.abspath('..'))

In [ ]:
import numpy as np
import torch
from tqdm.notebook import tqdm

from PIL import Image
import jpeg4py as jpeg

from src.dataset import PlatesCodeDataset
from src.lightning_module import OCRModule
from src.transforms import get_transforms
from src.predict_utils import matrix_to_string

In [ ]:
DEVICE = 'cuda:0'
VOCAB = '#&0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZÄÅÖÜĆČĐŠŽАБВГДЕЖЗИКЛМНОПРСТУФХЦЧШЭЮЯ'

In [ ]:
transforms = get_transforms(width=416, height=64, text_size=10, vocab=VOCAB, postprocessing=True, augmentations=False)

In [ ]:
dataset=PlatesCodeDataset(
    phase='test',
    data_folder='../data/',
    reset_flag=False,
    # transforms=transforms,
)

In [ ]:
module = OCRModule.load_from_checkpoint('../experiments/exp1/epoch_epoch=20-valid_string_match=0.965.ckpt')
_ = module.to(DEVICE)
_ = module.eval()

In [ ]:
gt_texts = []
pr_texts = []

for i in tqdm(range(len(dataset))):
    image, text, _, _ = dataset[i]

    transformed_image = transforms(image=image, text='')['image']
    predict = module(transformed_image[None].to(DEVICE)).cpu().detach()
    string_pred, _ = matrix_to_string(predict, VOCAB)

    gt_texts.append(text)
    pr_texts.append(string_pred[0])

gt_texts = np.array(gt_texts)
pr_texts = np.array(pr_texts)

In [ ]:
print(f'accuracy = {np.mean(gt_texts == pr_texts)}')

In [ ]:
# wrong indexes
np.where(gt_texts != pr_texts)[0]

In [ ]:
import random
random_idx = random.randint(0, len(dataset) - 1)
idx = random_idx
image, text, _, _ = dataset[idx]
transformed_image = transforms(image=image, text='')['image']
dummy_input = torch.rand(1, 3, 64, 416, device=DEVICE)
print(transformed_image[None].shape, dummy_input.shape)
print(f'pred = {pr_texts[idx]}')
print(f'true = {gt_texts[idx]}')
Image.fromarray(image)

In [ ]:
import onnxruntime as ort
import onnx

In [ ]:
# Convert to onnx
ONNX_MODEL_NAME = '../experiments/exp1/exp-1_plate-ocr-model.onnx'
DEVICE = 'cuda'
BATCH_SIZE = 1

dummy_input = torch.rand(1, 3, 64, 416, device=DEVICE)
torch.onnx.export(
    module,
    dummy_input,
    ONNX_MODEL_NAME,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes = {'input': [0], 'output': [0]}, # динамический батч, но можно и статический
)

In [ ]:
# check converted model
onnx_model = onnx.load(ONNX_MODEL_NAME)
onnx.checker.check_model(onnx_model)
print(onnx.helper.printable_graph(onnx_model.graph))

In [ ]:
# Init session

providers = [
    # 'CUDAExecutionProvider',
    'CPUExecutionProvider',
]

ort_session = ort.InferenceSession(
    ONNX_MODEL_NAME,
    providers=providers
)

print(f'{[input_.name for input_ in ort_session.get_inputs()]}')
print(f'{[output_.name for output_ in ort_session.get_outputs()]}')

In [ ]:
# prepare input tensor

BATCH_SIZE = 1
test_1 = '../data/dataset-plates/rus/test/img_140_20170102_080138_277_2215533.jpg'
test_2 = '../data/dataset-plates/rus_tl/test/6033578.jpg'
image = jpeg.JPEG(test_2).decode()
onnx_input = transforms(image=image, text='')['image'][None]
onnx_input = np.concatenate([onnx_input] * BATCH_SIZE)
print(onnx_input.shape)

ort_inputs = {ort_session.get_inputs()[0].name: onnx_input}
print(list(ort_inputs.keys()))

In [ ]:
# run ONNX Runtime inference
ort_outputs = ort_session.run(None, ort_inputs)[0]
print(ort_outputs.shape)

string_pred, _ = matrix_to_string(torch.from_numpy(ort_outputs), VOCAB)

In [ ]:
Image.fromarray(image)
string_pred